In [1]:
# import requirements
import os
import json
import sagemaker
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

# bucket_name_default = sagemaker_session.default_bucket()
# print(bucket_name_default)
# bucket_name = 'ml-demo-common'
%store -r bucket_name
print(bucket_name)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
execution role ARN: arn:aws:iam::629515838455:role/service-role/AmazonSageMaker-ExecutionRole-20231205T222162
sagemaker-gacheon-5


In [2]:
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')
print(training_data_uri)

s3://sagemaker-gacheon-5/data


In [3]:
!pygmentize 'train.py'

# 라이브러리 가져오기
import json
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import NearestNeighbors
import argparse
import joblib
import os


def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


def _parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument(
        "--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR")
    )
    parser.add_argument(
        "--data_dir",
        type=str,
        default=os.environ.get("SM_CHANNEL_DATA", "/opt/ml/input/data/training"),
    )

    return parser.parse_known_args()


def train_and_save_model(args):
    print("------ directory ----")
    # 현재 작업 디렉토리 내의 파일 목록 얻기
    file_list = os.listdir("/opt/ml/input/data/training")
    print(file_list)

    # 결과 출력
    print("현재 작업 디렉토리의 파일 및 디렉토리 목록:")
    for file in file_

In [4]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.m5.large'     # type of instance to use for training

training_script = 'train.py'

training_parameters = {
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}
FRAMEWORK_VERSION = "0.23-1"

In [5]:
from sagemaker.sklearn.estimator import SKLearn
estimator = SKLearn(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    hyperparameters=training_parameters,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
estimator.fit(training_data_uri)

INFO:sagemaker:Creating training-job with name: rf-scikit-2023-12-06-23-13-10-035


Using provided s3_resource
2023-12-06 23:13:10 Starting - Starting the training job...
2023-12-06 23:13:26 Starting - Preparing the instances for training......
2023-12-06 23:14:28 Downloading - Downloading input data...
2023-12-06 23:14:58 Training - Downloading the training image...
2023-12-06 23:15:33 Training - Training image download completed. Training in progress..2023-12-06 23:15:38,936 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-12-06 23:15:38,939 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-06 23:15:38,983 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-12-06 23:15:39,140 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-06 23:15:39,153 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-06 23:15:39,164 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpu

In [7]:
import boto3
estimator.latest_training_job.wait(logs="None")
sm_boto3 = boto3.client("sagemaker")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-12-06 23:15:59 Starting - Preparing the instances for training
2023-12-06 23:15:59 Downloading - Downloading input data
2023-12-06 23:15:59 Training - Training image download completed. Training in progress.
2023-12-06 23:15:59 Uploading - Uploading generated training model
2023-12-06 23:15:59 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-northeast-2-629515838455/rf-scikit-2023-12-06-23-13-10-035/output/model.tar.gz


In [8]:
from sagemaker.sklearn.model import SKLearnModel

source_dir = 'model'


# 사용자 정의 스크립트를 사용하는 경우 SKLearn Estimator를 생성
model = SKLearnModel(
    model_data=artifact,
    role=role,
    entry_point=training_script,
    framework_version=FRAMEWORK_VERSION,
)



predictor = model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    endpoint_name='gcu-ml1-steam-game-final-endpoint',
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-12-06-23-16-28-682
INFO:sagemaker:Creating endpoint-config with name gcu-ml1-steam-game-final-endpoint
INFO:sagemaker:Creating endpoint with name gcu-ml1-steam-game-final-endpoint


----!

In [9]:
import json
import sagemaker
import pandas as pd

game_info = np.load("ml-latest-small/s3/games.npy", allow_pickle=True)

game_info = pd.DataFrame(game_info)


result = predictor.predict([game_info.iloc[20]])
print(result)

[108.8145933]
